In [2]:
import pandas as pd

import io 
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from pdfminer.converter import TextConverter
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfpage import PDFPage

import pdfplumber

import requests

/home/noahdarwich/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (5.1.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [3]:
import tempfile 
from oauth2client.service_account import ServiceAccountCredentials

from googleapiclient.http import MediaIoBaseDownload

In [4]:
from arabic_reshaper import ArabicReshaper
from bidi.algorithm import get_display

In [32]:
from pdfquery import PDFQuery
from io import BytesIO, StringIO
from pdfquery import PDFQuery
from PIL import Image
import pytesseract
from pdfminer.high_level import extract_text_to_fp

from pdfminer.layout import LAParams

from PyPDF2 import PdfFileReader, PdfFileWriter
import pyarabic.araby as araby

import camelot

In [3]:
# df = pd.read_csv('Full_df.csv')
# df

In [6]:
def extract_text_from_pdf(pdf_content):
  resource_manager = PDFResourceManager()
  fake_file_handle = io.StringIO()
  converter = TextConverter(resource_manager, fake_file_handle)
  page_interpreter = PDFPageInterpreter(resource_manager, converter)

  with io.BytesIO(pdf_content) as fh:
    for page in PDFPage.get_pages(fh,
                                  caching=True,
                                  check_extractable=True):
      page_interpreter.process_page(page)

    text = fake_file_handle.getvalue()

  # close open handles
  converter.close()
  fake_file_handle.close()

  return text

In [ ]:
# text_ls = [] 
# for link in pdf_links:
#     # download the PDF
#     pdf_url = link['href']
    
#     pdf_content = pdf_response.content
#     # extract the text from the PDF
#     text = extract_text_from_pdf(pdf_content)
#     text_ls.append(text)
    
# print(soup)

In [20]:
# url = df.iloc[0]['pdf_link']
# pdf_response = requests.get(url)
# pdf_content = pdf_response.content
# text = extract_text_from_pdf(pdf_content)
# text

'ةحفص 32ةريل 10002019  لولأا نوناك  2  نينثلااةرشع ةعبارلا  ةنسلا  3922  ددعلاlundi  2  Décembre  2019  no 3922   14ème annéeةبهو دمحم ●يداصتقا جذومنةعارزلا لتقل مّمُصهللا لضف ميلحلا دبع ●جذومنلا ةفارخ نعلّجؤم رايهنا ةياكحولاملا سأر »يزكرملا« طايتحاىشلاتي رلاودلاب16نميلا ٌيدوعس ٌطغض ةقطنم ءاشنلإةلزاع)ب ف أ(]15  ـ 14[ ىضوفلاقارعلا برضت]2[  اًيموكح مساح عوبسأفتنشم يليإ ىقبي نأ ىنمتنةبعللا ىقبتل دلبلاةلباقم8 زاغو ايبيلو ايكرتطّسوتملا!»ةبعللا تأدب نلآا«ةيضق18\x0c23نانبلنانبلفلم ،سانلل ةنماض ةسسؤمك ةلودلا عجارتو ،ةريبكلا بازحلأا طوقس ةريثك تائف تلعج ،ةّداح كلاهتسا ةموظنم ىلإ داصتقلاا لّوحتو ،مهميلعتل ريثكلا مهتلائاع تقفنأ نيذلا تاعماجلا يجيّرخ نم لبرش لوقي امك ،نانبل .لخدلل فلتخم ردصم هاجتا يف نوبهذي يف لا ،ةيرشبلا تاقاطلا ريدصت يف اًصّصختم راص ،ساحن عّزوم ،انه بابشلا نم يقب نمو .مهتامدخ وأ مهتاجتنم ريدصت اًصوصخ ،ةيدجلا بازحلأا يف ينطرخنلما ةئف :ةدع تائف ينب تاعاطق يف جاتنلإا ةليلق فئاظو ىلع لصح ءزجو ،هللا بزح ىلع شاتعي ثلاث ءزجو .عاطقلإاو بازحلأا ةرطيس لعفب ةلودلا ةوخلإا هلسري ام ضعب وأ ةي

In [7]:
creds = ServiceAccountCredentials.from_json_keyfile_name('leba-375419-92c3baf53205.json',
                                                         scopes=['https://www.googleapis.com/auth/drive'])
service = build('drive', 'v3', credentials=creds)
folder_id = '1BVzI84DDDtbvU3pp1k0yERF5YKIs84R0'
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()

files = results.get("files", [])
next_page_token = results.get("nextPageToken")

while next_page_token:
    results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)", pageToken=next_page_token).execute()
    files.extend(results.get("files", []))
    next_page_token = results.get("nextPageToken")
    
# Create new folder "keywords" inside "AAA"
folder_metadata = {'name': 'events', 'parents': [folder_id], 'mimeType': 'application/vnd.google-apps.folder'}
folder = service.files().create(body=folder_metadata).execute()
events_folder_id = folder.get('id')

In [17]:
results = service.files().list(q=f"'{folder_id}' in parents",fields="nextPageToken, files(id, name)").execute()
files = results.get('files', [])

# Iterate through the files
for file in files:
    file_id = file.get('id')
    file_name = file.get('name')
    request = service.files().export_media(fileId=file_id,mimeType='application/pdf')
    fh = io.BytesIO()
    downloader = MediaIoBaseDownload(fh, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Download %d%%." % int(status.progress() * 100))
    fh.seek(0)
    with open("downloaded_file.pdf", 'wb') as f:
        content = fh.read()
        print(content)

HttpError: <HttpError 403 when requesting https://www.googleapis.com/drive/v3/files/1wNpWF4CXEFRqHSgPWuGaftdW4027PFjn/export?mimeType=application%2Fpdf&alt=media returned "Export only supports Docs Editors files.". Details: "[{'domain': 'global', 'reason': 'fileNotExportable', 'message': 'Export only supports Docs Editors files.'}]">

In [23]:
df_results = pd.read_csv('df_results.csv', index_col=0)
df_results

,issue,url,date,pdf_link,status
0,العدد 3922,https://www.al-akhbar.com/Editions/2019/12/2/PDF,2019-12-2,https://www.al-akhbar.com/PDF_Files/3922/alakh...,NaN
1,العدد 3923,https://www.al-akhbar.com/Editions/2019/12/3/PDF,2019-12-3,NaN,not_saved
2,العدد 3924,https://www.al-akhbar.com/Editions/2019/12/4/PDF,2019-12-4,https://www.al-akhbar.com/PDF_Files/3924/alakh...,NaN
3,العدد 3925,https://www.al-akhbar.com/Editions/2019/12/5/PDF,2019-12-5,https://www.al-akhbar.com/PDF_Files/3925/alakh...,NaN
4,العدد 3926,https://www.al-akhbar.com/Editions/2019/12/6/PDF,2019-12-6,https://www.al-akhbar.com/PDF_Files/3926/alakh...,NaN
...,...,...,...,...,...
2923,العدد 1028,https://www.al-akhbar.com/Editions/2010/1/26/PDF,2010-1-26,https://www.al-akhbar.com/PDF_Files/1028/alakh...,saved
2924,العدد 1029,https://www.al-akhbar.com/Editions/2010/1/27/PDF,2010-1-27,https://www.al-akhbar.com/PDF_Files/1029/alakh...,saved
2925,العدد 1030,https://www.al-akhbar.com/Editions/2010/1/28/PDF,2010-1-28,https://www.al-akhbar.com/PDF_Files/1030/alakh...,saved
2926,العدد 1031,https://www.al-akhbar.com/Editions/2010/1/29/PDF,2010-1-29,https://www.al-akhbar.com/PDF_Files/1031/alakh...,saved


In [ ]:

# # search for specific keywords in the content
# if 'keyword1' in content or 'keyword2' in content:
#     # move the file to the "keywords" folder
#     file_metadata = {'parents': [events_folder_id]}
#     service.files().update(fileId=file_id, body=file_metadata).execute()


In [ ]:
configuration = {
    'delete_harakat': True,
    'delete_tatweel ': True,
}

pdf_response = requests.get(url)
text = pdf_response.content

In [41]:
reshaper = ArabicReshaper(configuration=configuration)
corrected_text = reshaper.reshape(text)
corrected_text = get_display(text)
corrected_text = corrected_text.replace('ـ', '')
corrected_text

'\x0cأنجل بوليغان  المكسيكواالستهدافات الخارجية.تعج بالصراعات واالنقسامات البتة مع موقعنا في منطقة الذي تقوقعنا فيه ال يتناسب الذي بنيناه والنموذج النقدي االقتصاد المفتوح والهشّ عن خرافة النموذج... وحكاية انهيار مؤجّلالحت بوادره، ونأمل أن ال تكتمل فصوله.الخيارات ومواجهة استحقاقات ما بعد االنهيار الذي هي معيارنا الرئيسي في وضع السياسات وتحديد والبشري في هذه املرحلة االنتقالية الصعبة والطويلة العموم، يجب أن تكون متطلبات األمن االجتماعي الشركات واملؤسسات التابعة للقطاع العام(. وعلى الخزينة التي يحملها الصندوق ب»حصص« في بعض االجتماعي )يمكن مثاً استبدال جزء من سندات نهاية الخدمة في الصندوق الوطني للضمان املستفيدين منها، وحماية القيمة الحقيقية لتعويضات وزيادة موازنات دعم األسر األكثر فقرًا وتوسيع قاعدة مؤقتة لضمان البطالة التي ستتضاعف أرقامها، االجتماعية لألزمة، تتضمن مثاً استحداث برامج الوفر املتحقق من هذه الهندسة في استيعاب التبعات الدين العام وتخفيف خدمته، على أن يستعمل جزء من ضخمة ومدروسة ومتوازنة ملصلحة الدولة لتقليص الدائنة واملدينة خفضًا ملموسًا، وإجراء هندسة مالية املفروضة على التحوي

In [26]:
corrected_text.replace('ـ','')
corrected_text

'\x0cأنجل بوليغان ــ المكسيكواالستهدافات الخارجية.تعج بالصراعات واالنقسامات البتة مع موقعنا في منطقة الذي تقوقعنا فيه ال يتناسب الذي بنيناه والنموذج النقدي االقتصاد المفتوح والهشّ عن خرافة النموذج... وحكاية انهيار مؤجّلالحت بوادره، ونأمل أن ال تكتمل فصوله.الخيارات ومواجهة استحقاقات ما بعد االنهيار الذي هي معيارنا الرئيسي في وضــع السياسات وتحديد والبشري في هذه املرحلة االنتقالية الصعبة والطويلة الــعــمــوم، يــجــب أن تــكــون متطلبات األمـــن االجــتــمــاعــي الــشــركــات واملــؤســســات التابعة للقطاع الــعــام(. وعلى الخزينة التي يحملها الصندوق بـ»حصص« في بعض االجــتــمــاعــي )يــمــكــن مــثــاً اســتــبــدال جـــزء مــن ســنــدات نـــهـــايـــة الــــخــــدمــــة فــــي الــــصــــنــــدوق الــــوطــــنــــي لــلــضــمــان املستفيدين منها، وحماية القيمة الحقيقية لتعويضات وزيادة موازنات دعم األسر األكثر فقرًا وتوسيع قاعدة مــؤقــتــة لــضــمــان الــبــطــالــة الــتــي ســتــتــضــاعــف أرقــامــهــا، االجتماعية لــألزمــة، تتضمن مــثــاً اســتــحــداث برامج الوفر املتحقق من ه

In [24]:
test_sample = df_results[0:5]
test_folder_id = '19Nx9P1ekaDyoX06z2sj0FKokfcskouNc'

In [26]:
url = test_sample.iloc[0]['pdf_link']
url

'https://www.al-akhbar.com/PDF_Files/3922/alakhbar20191202.pdf'

In [76]:
response = requests.get(url)

with open('file.pdf', 'wb') as f:
    f.write(response.content)

with open('file.pdf', 'rb') as f:
    
    pdf = PdfFileReader(f)
    laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    outfp = StringIO()

    for page in range(0, pdf.getNumPages()+1):
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        corrected_text = reshaper.reshape(text)
        corrected_text = get_display(text)
        #print(corrected_text)
        if 'أغاثا كريستي'in corrected_text:
            output_pdf = PdfFileWriter()
            output_pdf.addPage(pdf.getPage(page))
            with open('single_page.pdf', 'wb') as f:
                output_pdf.write(f)

ValueError: seek of closed file

In [60]:

pdf_response = requests.get(url)

with open('file.pdf', 'wb') as f:
    f.write(pdf_response.content)

with open('file.pdf', 'rb') as f:
    # pdf = PDFQuery(f)
    # pdf.load()
    # blocks = pdf.extract([
    #     ('with_parent','LTPage'),
    #     ('block_txt','LTTextLineHorizontal')
    # ])
    # for block in blocks:
    #     print(block.text.strip())
    pdf = PdfFileReader(f)

    outfp = StringIO()
    laparams = LAParams(char_margin=1.0, line_margin=0.5, word_margin=0.1)
    
    for page in range(1,pdf.getNumPages()):
        outfp.seek(0)
        extract_text_to_fp(f, outfp, laparams=laparams, page_numbers={page})
        outfp.seek(0)
        text = outfp.read()
        reshaper = ArabicReshaper(configuration=configuration)
        #corrected_text = reshaper.reshape(text)
        corrected_text = get_display(text)
        print(corrected_text)
        if 'أسبوع' in corrected_text:
            print('TADAAAAAAAAAAAAAAA',page)
            break


3

لبنان

اإلثنين  2  كانون األول  2019  العدد  3922

عت 

ر وخط

قة بأسماء هذه الجمعيات 

م وفرص العمل والدور القيادي، ورعاية 

وما الذي تملكه اآلن، وصورة مبسّ طة عن نظام حياتها العامة، 
مثل أين تعيش وأين يتعلم األبناء وأين يقضون اإلجازات ونوع 
السيارات والخدمات التي يحصلون عليها في حياتهم اليومية؟
، من أين يُ نفق حزب الكتائب طوال العقود املاضية، وما هي 
مثال
املهن التي يقوم بها آل الجميل بكل فروعهم؟ أو لنسأل حزب 
فقراء املسيحيني، أي القوات اللبنانية، عن مصدر نفقاته الشهرية 
والسنوية في كل القطاعات، بما في ذلك كلفة حياة قائده 
وخدماته وحراسته. واألمر نفسه بالنسبة إلى التيار الوطني 
الحر وحركة أمل، والزعيم الوطني الكبير املفدّ ى وليد جنبالط، 
وحجم اإلنفاق القائم على الدولة لحراستهم وتأمني خدماتهم 
اللوجستية؟. وماذا عن بقية الزعامات، من بقايا اإلقطاع 
األسعدي في الجنوب، إلى مراعبة عكار، مرورًا بآل فرنجية 
ومعوض والخازن وفرعون والصحناوي، حتى آل الحريري في 
هذه األيام...؟
ماذا عن كل جمعيات دعم املرأة وتمكينها وحريتها ورفض 
تعنيفها، ومقاومة التمييز الجندري، وحق األمومة والجنسية، 
واملساواة

In [34]:
url

'https://www.al-akhbar.com/PDF_Files/3922/alakhbar20191202.pdf'